***Data Pre-Processor***


In [17]:
%run DeepFake_Dataloader.py
%run utils.py

In [18]:
import h5py
import torch
import numpy as np

# Pretty Print
import pprint
pp = pprint.PrettyPrinter(width=20)

In [19]:
# batch_sz = 128 
batch_sz = 10
image_folder_path='/home/jupyter/image_folder_256'

dataloader = get_preprocessors(
    image_root=image_folder_path, 
    output_size=363,
    batch_size=32,
    num_workers=8,
    destf_hdf5='/content/LSUN_Cars.hdf5',
#     transforms=convert_to_256
)

total_imgs  = len(dataloader.dataset.samples)
data_origin = dataloader.dataset.class_to_idx
data_origin.update({y:x for x,y in data_origin.items()})
print(dataloader.dataset)
print('Classes:')
pp.pprint(dataloader.dataset.classes)
print(data_origin)

/home/jupyter/image_folder_256
Dataset DeepFakePreProcessor
    Number of datapoints: 40002
    Root location: /home/jupyter/image_folder_256
Classes:
['lsun_cats',
 'lsun_churches',
 'stylegan2_cats',
 'stylegan2_churches']
{'lsun_cats': 0, 'lsun_churches': 1, 'stylegan2_cats': 2, 'stylegan2_churches': 3, 0: 'lsun_cats', 1: 'lsun_churches', 2: 'stylegan2_cats', 3: 'stylegan2_churches'}


In [ ]:
progress = ProgressMonitor(length=total_imgs)
rad_len  = 182
hdf5_fn  = 'LSUN_BCCC_256.hdf5'
with h5py.File(hdf5_fn, 'w') as f:
    """
        Create HDF5 datasets 
    """
    data_dset = f.create_dataset('fft_data',  (total_imgs, rad_len), dtype='f')
    lbl_dset  = f.create_dataset('lbl_data',  (total_imgs,),     dtype='i')
    orgn_dset = f.create_dataset('orgn_data', (total_imgs,),     dtype='i')
    img_size  = f.create_dataset('img_size',  (total_imgs,),     dtype='i')
    data_size = f.create_dataset('data_size', (total_imgs,),     dtype='i')
    
    for i, (img_data, target, origin_dataset, _, _) in enumerate(dataloader):
        start_idx = i * batch_sz
        for j in range(img_data.shape[0]):
            idx = start_idx + j
            progress.update(1, 0)
            if (target[j] > 1):
                print(target[j])
            data_dset[idx] = img_data[j]
            lbl_dset[idx]  = target[j]
            orgn_dset[idx] = data_origin[origin_dataset[j]]
            img_size[idx]  = data_img_size[origin_dataset[j]]
            data_size[idx] = rad_size[origin_dataset[j]]
        

In [ ]:
import matplotlib.pyplot as plt

with h5py.File(hdf5_fn, 'r') as f:
    dset = f['fft_data']
    plt.plot(dset[200])
    for i in range(10):
        plt.plot(dset[np.random.randint(20000)])
#         plt.show()
    plt.show()

In [ ]:
with h5py.File(hdf5_fn, 'r') as f:
    dset = f['lbl_data']
    for i in range(len(dset)):
        if dset[i] != 1 and dset[i] != 0:
            print(dset[i])

In [ ]:
!gcloud auth activate-service-account --key-file ./playground-270702-4d2df0765336.json

In [ ]:
# auth.authenticate_user()

project_id = 'playground-270702'
!gcloud config set project {project_id}
!gsutil ls

bucket_name = 'img-hosting-bucket'
!gsutil -m cp -r /home/jupyter/CSE253_FinalProject/LSUN_BCCC_256.hdf5 gs://{bucket_name}/

In [ ]:
!wget https://storage.googleapis.com/img-hosting-bucket/LSUN_BCCC_256.hdf5

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec, colors
import os
from PIL import Image
import random
from torchvision import transforms
import numpy as np
import scipy.signal as signal

b, a = signal.butter(8, 0.125)


file_name = './imgs/class_1'

(img_data, target, origin_dataset, img, ms_img) = next(iter(dataloader))
for i, dset in enumerate(origin_dataset):
    if dset == 'lsun_cars':
        print(dset)
        img_data = img_data[i]
        img      = img[i]
        origin_dataset = origin_dataset[i]
        target   = target[i]
        ms_img   = ms_img[i]
        break

sg2_img = transforms.ToPILImage()(img.permute(2, 0, 1))
display(sg2_img)
sg2_img.save('stylegan2_cars.png')

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex='col',
                        gridspec_kw={'hspace': 1, 'wspace': 0.15, 'width_ratios': [1, 1, 2]},
                        figsize=(20, 5))
fig.suptitle('Image from ' + origin_dataset, fontsize=20)

ax1.axis('off')
ax2.axis('off')

ax1.set_title('Input Image',           size=15)
ax2.set_title('Amplitude Spectrum 2D', size=15)
ax3.set_title('1D Power Spectrum',     size=15)

ax1.imshow(img)
print(ms_img.max())
print(ms_img.min())
ax2.imshow(ms_img, cmap='Greys', norm=colors.Normalize(vmin=ms_img.min(), vmax=ms_img.max()))

plt.xlabel('Spatial Frequency', fontsize=12)
plt.ylabel('Power Spectrum',    fontsize=12)
ax3.plot(img_data)

plt.savefig(file_name + '.jpg', dpi=300)
plt.show()

y = signal.filtfilt(b, a, ms_img, padlen=150)

plt.imshow(y)
plt.figure()
plt.imshow(ms_img)

In [9]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

print(mcolors.BASE_COLORS)
print(mcolors.BASE_COLORS.keys())
print(list(mcolors.BASE_COLORS.keys())[0])
print(mcolors.TABLEAU_COLORS)
for k, v in mcolors.TABLEAU_COLORS.items():
    print(k, v)

{'b': (0, 0, 1), 'g': (0, 0.5, 0), 'r': (1, 0, 0), 'c': (0, 0.75, 0.75), 'm': (0.75, 0, 0.75), 'y': (0.75, 0.75, 0), 'k': (0, 0, 0), 'w': (1, 1, 1)}
dict_keys(['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'])
b
OrderedDict([('tab:blue', '#1f77b4'), ('tab:orange', '#ff7f0e'), ('tab:green', '#2ca02c'), ('tab:red', '#d62728'), ('tab:purple', '#9467bd'), ('tab:brown', '#8c564b'), ('tab:pink', '#e377c2'), ('tab:gray', '#7f7f7f'), ('tab:olive', '#bcbd22'), ('tab:cyan', '#17becf')])
tab:blue #1f77b4
tab:orange #ff7f0e
tab:green #2ca02c
tab:red #d62728
tab:purple #9467bd
tab:brown #8c564b
tab:pink #e377c2
tab:gray #7f7f7f
tab:olive #bcbd22
tab:cyan #17becf


In [10]:
c = np.diag(np.arange(1, 10))
c[0] += 1
print(c)
print(np.mean(c, axis=0))
print(np.std(c, axis=0))

[[2 1 1 1 1 1 1 1 1]
 [0 2 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 6 0 0 0]
 [0 0 0 0 0 0 7 0 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 0 0 0 0 9]]
[0.22222222 0.33333333 0.44444444 0.55555556 0.66666667 0.77777778
 0.88888889 1.         1.11111111]
[0.62853936 0.66666667 0.95581392 1.25707872 1.56347192 1.87247773
 2.18298697 2.49443826 2.80651799]
